In [1]:
import openai

client = openai.OpenAI()  # 直接建立 client，不用傳 api_key

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "你好"}]
)

print(response.choices[0].message.content)

您好！有什么可以帮助您的吗？


In [3]:
import json

# Read the JSON file
json_path = "訓練.json"

with open(json_path, 'r', encoding='utf-8') as file:
    training_data = json.load(file)

# Display the first 3 entries
print("First 3 entries from the training data:")
for i, entry in enumerate(training_data[:3]):
    print(f"\nEntry {i+1}:")
    for key, value in entry.items():
        print(f"{key}:")
        print(f"{value}")
    print("-" * 50)

First 3 entries from the training data:

Entry 1:
instruct:
參考資料：
1. 元太科技公布首季營收年增25%，主要受惠電子紙顯示器訂單大幅成長。
2. 全球電子標籤應用普及，歐洲與日本客戶拉貨動能強勁。
3. 公司預期今年營收將持續成長雙位數。

問題：我最近看到元太漲不少，現在進場是不是風險比較高？
output:
元太受惠電子紙應用擴大，成長動能確實強勁。不過短線已反映部分利多，建議投資人可觀察是否有技術面拉回後的買點，再分批進場降低風險。
--------------------------------------------------

Entry 2:
instruct:
參考資料：
1. 華碩第一季筆電出貨量略低於預期，但AI PC新品即將上市。
2. 公司強調AI筆電未來將成為營收主力。
3. 外資報告多給予「觀望」評等，認為轉型初期仍具不確定性。

問題：華碩轉型AI PC有搞頭嗎？可以買進嗎？
output:
華碩推動AI PC是長期正向策略，但短期營收與市佔仍待驗證。建議投資人可視個人風險承受度小額試單，並關注實際出貨與市占率變化後再決定是否加碼。
--------------------------------------------------

Entry 3:
instruct:
參考資料：
1. 長榮航空公布第一季獲利創新高，主要因機票價格高檔與貨運回溫。
2. 第二季中國與東南亞航線需求強勁。
3. 燃油成本仍具不確定性，需關注國際油價波動。

問題：我現在進場買長榮航，會太晚嗎？
output:
長榮航獲利動能佳，短線仍有上漲空間。不過航運股波動性高，建議分批進場並關注國際油價與載客率變化，以利動態調整部位。
--------------------------------------------------


In [10]:
import openai
import json
import time
import os

# 檔案名稱
input_file = '訓練.json'
output_file = '訓練_updated.json'
temp_file = '訓練_temp.json'

# 讀取現有訓練資料
if os.path.exists(temp_file):
    print("🔄 偵測到未完成的 temp 檔案，繼續補完...")
    with open(temp_file, 'r', encoding='utf-8') as file:
        training_data = json.load(file)
else:
    with open(input_file, 'r', encoding='utf-8') as file:
        training_data = json.load(file)

# 主題列表
topics = [
    "台積電3奈米製程量產",
    "鴻海電動車進度",
    "聯發科新一代晶片",
    "緯創印度廠擴產",
    "群創面板報價走勢",
    "南亞科記憶體出貨",
    "華碩AI PC布局",
    "廣達伺服器訂單",
    "泰鼎PCB產能",
    "台達電充電樁業務"
]

# Function schema
function_def = {
    "name": "generate_stock_advice",
    "description": "根據主題生成符合投資建議格式的JSON資料",
    "parameters": {
        "type": "object",
        "properties": {
            "instruct": {
                "type": "string",
                "description": "投資建議的問題與參考資料"
            },
            "output": {
                "type": "string",
                "description": "根據參考資料與問題產生的建議回答"
            }
        },
        "required": ["instruct", "output"]
    }
}

client = openai.OpenAI()
new_entries = []

# 從尚未產生的主題開始
existing_topics = {entry["instruct"] for entry in training_data if "instruct" in entry}
remaining_topics = [t for t in topics if all(t not in et for et in existing_topics)]

for idx, topic in enumerate(remaining_topics):
    try:
        user_prompt = f"""根據主題「{topic}」，請產生一則股市投資建議，包含以下結構：
- instruct 欄位：要含三點參考資料（具體數據或事實），以及一個和資料內容有關的投資決策問題。
- output 欄位：要有對股價的預測與具體建議，長度 80-100 字。

請用 function calling 格式回傳。"""

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是資深股市分析師，請精準地根據資料生成投資建議。"},
                {"role": "user", "content": user_prompt}
            ],
            functions=[function_def],
            function_call={"name": "generate_stock_advice"},
            temperature=0.7
        )

        # 提取結果
        response_args = response.choices[0].message.function_call.arguments
        result = json.loads(response_args)
        new_entries.append(result)
        training_data.append(result)

        print(f"✅ 已完成：{topic}")

        # 每3筆儲存一次暫存檔
        if len(new_entries) % 3 == 0 or idx == len(remaining_topics) - 1:
            with open(temp_file, 'w', encoding='utf-8') as file:
                json.dump(training_data, file, ensure_ascii=False, indent=2)
            print("💾 已暫存資料至訓練_temp.json")

        time.sleep(1)

    except Exception as e:
        print(f"⚠️ 生成「{topic}」時發生錯誤: {str(e)}")

# 最後一次正式儲存
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(training_data, file, ensure_ascii=False, indent=2)

# 刪除 temp 檔
if os.path.exists(temp_file):
    os.remove(temp_file)

# 顯示結果
print("\n✅ 所有新生成資料：")
for entry in new_entries:
    print(json.dumps(entry, ensure_ascii=False, indent=2))


✅ 已完成：台積電3奈米製程量產
✅ 已完成：鴻海電動車進度
✅ 已完成：聯發科新一代晶片
💾 已暫存資料至訓練_temp.json
✅ 已完成：緯創印度廠擴產
✅ 已完成：群創面板報價走勢
✅ 已完成：南亞科記憶體出貨
💾 已暫存資料至訓練_temp.json
✅ 已完成：華碩AI PC布局
✅ 已完成：廣達伺服器訂單
✅ 已完成：泰鼎PCB產能
💾 已暫存資料至訓練_temp.json
✅ 已完成：台達電充電樁業務
💾 已暫存資料至訓練_temp.json

✅ 所有新生成資料：
{
  "instruct": "1. 台積電宣布成功開發3奈米製程技術。\n2. 3奈米製程有望提升晶片效能並節省能源。\n3. 市場預期台積電將加速3奈米製程量產。\n投資決策問題：你認為3奈米製程將如何影響台積電的競爭力？",
  "output": "預測台積電將因3奈米製程技術的成功開發而受益。這將提升公司在晶片市場的競爭力，預計股價將隨著3奈米製程量產進展而逐步上升。建議投資者留意相關消息，可考慮在適合時機進行投資。"
}
{
  "instruct": "1. 鴻海正積極推動電動車產業，計劃在未來幾年內推出多款電動車型。\n2. 鴻海與多家車廠合作開發電動車技術，包括充電系統、電池技術等。\n3. 鴻海電動車產業的投入與市場反應將直接影響公司營收及股價表現。\n投資決策問題：鴻海的電動車業務對公司未來發展的影響如何？",
  "output": "鴻海積極參與電動車產業，預計將帶動公司營收增長。基於電動車市場潛力，預測鴻海股價將持續走高。建議投資者密切關注鴻海電動車相關進度，可考慮長期持有鴻海股票以享受成長紅利。"
}
{
  "instruct": "1. 聯發科推出新一代晶片，預計搭載更多先進功能。\n2. 市場需求持續增長，智慧型裝置市場競爭激烈。\n3. 聯發科近期營收穩健成長，技術研發投入積極。是否應增加聯發科股票持有量？",
  "output": "基於聯發科新一代晶片的應用前景以及市場需求，預計聯發科股價將持續上升。建議投資者在短期內增加聯發科股票持有量，以參與智慧型裝置市場的競爭激烈中取得更多利潤。盤整後或許可考慮部分賣出以獲利。"
}
{
  "instruct": "1. 緯創宣布在印度擴大生產，預計增加產能50%\n2. 印度市場需求增加，競爭優勢

In [14]:
import openai
import json
import time
import os

# 檔案名稱
input_file = '訓練.json'
output_file = '訓練_updated.json'
temp_file = '訓練_temp.json'

# 讀取現有訓練資料
if os.path.exists(temp_file):
    print("🔄 偵測到未完成的 temp 檔案，繼續補完...")
    with open(temp_file, 'r', encoding='utf-8') as file:
        training_data = json.load(file)
else:
    with open(input_file, 'r', encoding='utf-8') as file:
        training_data = json.load(file)

# ========== ⬇️ 自動擴增主題清單區塊 ========== #
# 各產業代表性公司
base_topics = {
    "科技": ["台積電", "鴻海", "聯發科", "緯創", "群創", "華碩", "廣達", "南亞科", "技嘉"],
    "金融": ["兆豐金", "中信金", "玉山金", "台新金", "富邦金", "國泰金"],
    "傳產": ["台泥", "亞泥", "榮成", "正隆", "中鋼", "中鴻"],
    "消費": ["統一", "全家", "大潤發", "味全", "卜蜂", "大成"],
    "能源": ["中油", "台電", "台塑化", "亞東石化"],
    "航運": ["長榮", "陽明", "萬海", "華航", "長榮航"],
    "生技": ["中裕", "台康生技", "國光生", "合一", "生華科"]
}

# 各時間與區域變化
time_variants = ["2025年展望", "Q1財報", "Q2財報", "Q3財報", "Q4財報"]
region_variants = ["美國市場", "中國布局", "越南投資", "歐洲拓展", "台灣擴產"]

# 技術分析情境
tech_analysis_variants = [
    "突破季線", "跌破月線", "MACD黃金交叉", "KD指標高檔鈍化",
    "外資連五日買超", "融資餘額下降", "短線爆量上漲", "籌碼集中度提高"
]

# 擴增主題列表
topics = []

for sector, companies in base_topics.items():
    for company in companies:
        for time_v in time_variants:
            topics.append(f"{company}{time_v}")
        for region_v in region_variants:
            topics.append(f"{company}{region_v}")
        for tech_v in tech_analysis_variants:
            topics.append(f"{company}{tech_v}")

# ========== ⬆️ 擴增主題區塊結束 ========== #

# Function schema
function_def = {
    "name": "generate_stock_advice",
    "description": "根據主題生成符合投資建議格式的JSON資料",
    "parameters": {
        "type": "object",
        "properties": {
            "instruct": {
                "type": "string",
                "description": "投資建議的問題與參考資料"
            },
            "output": {
                "type": "string",
                "description": "根據參考資料與問題產生的建議回答"
            }
        },
        "required": ["instruct", "output"]
    }
}

client = openai.OpenAI()
new_entries = []

# 過濾已存在的主題
existing_topics = {entry["instruct"] for entry in training_data if "instruct" in entry}
remaining_topics = [t for t in topics if all(t not in et for et in existing_topics)]

# 開始逐筆生成
for idx, topic in enumerate(remaining_topics):
    try:
        user_prompt = f"""根據主題「{topic}」，請產生一則股市投資建議，包含以下結構：
- instruct 欄位：要含三點或以上的參考資料（具體數據或事實），以及一個和資料內容有關的投資決策問題。
- output 欄位：要有對股價的預測與具體建議，長度 80-100 字。

請用 function calling 格式回傳。"""

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是資深股市分析師，請精準地根據資料生成投資建議。"},
                {"role": "user", "content": user_prompt}
            ],
            functions=[function_def],
            function_call={"name": "generate_stock_advice"},
            temperature=0.7
        )

        # 提取 function call 結果
        response_args = response.choices[0].message.function_call.arguments
        result = json.loads(response_args)
        new_entries.append(result)
        training_data.append(result)

        print(f"✅ 已完成：{topic}")

        # 每3筆儲存一次暫存檔
        if len(new_entries) % 3 == 0 or idx == len(remaining_topics) - 1:
            with open(temp_file, 'w', encoding='utf-8') as file:
                json.dump(training_data, file, ensure_ascii=False, indent=2)
            print("💾 已暫存資料至訓練_temp.json")

        time.sleep(1)

    except Exception as e:
        print(f"⚠️ 生成「{topic}」時發生錯誤: {str(e)}")

# 儲存正式結果
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(training_data, file, ensure_ascii=False, indent=2)

if os.path.exists(temp_file):
    os.remove(temp_file)

# 顯示結果
print("\n✅ 所有新生成資料：")
for entry in new_entries:
    print(json.dumps(entry, ensure_ascii=False, indent=2))


✅ 已完成：台積電2025年展望
✅ 已完成：台積電Q1財報
✅ 已完成：台積電Q2財報
💾 已暫存資料至訓練_temp.json
✅ 已完成：台積電Q3財報
✅ 已完成：台積電Q4財報
✅ 已完成：台積電美國市場
💾 已暫存資料至訓練_temp.json
✅ 已完成：台積電中國布局
✅ 已完成：台積電越南投資
✅ 已完成：台積電歐洲拓展
💾 已暫存資料至訓練_temp.json
✅ 已完成：台積電台灣擴產
✅ 已完成：台積電突破季線
✅ 已完成：台積電跌破月線
💾 已暫存資料至訓練_temp.json
✅ 已完成：台積電MACD黃金交叉
✅ 已完成：台積電KD指標高檔鈍化
✅ 已完成：台積電外資連五日買超
💾 已暫存資料至訓練_temp.json
✅ 已完成：台積電融資餘額下降
✅ 已完成：台積電短線爆量上漲
✅ 已完成：台積電籌碼集中度提高
💾 已暫存資料至訓練_temp.json
✅ 已完成：鴻海2025年展望
✅ 已完成：鴻海Q1財報
✅ 已完成：鴻海Q2財報
💾 已暫存資料至訓練_temp.json
✅ 已完成：鴻海Q3財報
✅ 已完成：鴻海Q4財報
✅ 已完成：鴻海美國市場
💾 已暫存資料至訓練_temp.json
✅ 已完成：鴻海中國布局
✅ 已完成：鴻海越南投資
✅ 已完成：鴻海歐洲拓展
💾 已暫存資料至訓練_temp.json
✅ 已完成：鴻海台灣擴產
✅ 已完成：鴻海突破季線
✅ 已完成：鴻海跌破月線
💾 已暫存資料至訓練_temp.json
✅ 已完成：鴻海MACD黃金交叉
✅ 已完成：鴻海KD指標高檔鈍化
✅ 已完成：鴻海外資連五日買超
💾 已暫存資料至訓練_temp.json
✅ 已完成：鴻海融資餘額下降
✅ 已完成：鴻海短線爆量上漲
✅ 已完成：鴻海籌碼集中度提高
💾 已暫存資料至訓練_temp.json
✅ 已完成：聯發科2025年展望
✅ 已完成：聯發科Q1財報
✅ 已完成：聯發科Q2財報
💾 已暫存資料至訓練_temp.json
✅ 已完成：聯發科Q3財報
✅ 已完成：聯發科Q4財報
✅ 已完成：聯發科美國市場
💾 已暫存資料至訓練_temp.json
✅ 已完成：聯發科中國布局
✅ 已完成：聯發科越南投資
✅ 已完成：聯發科歐洲拓展
💾 已暫存資料至訓練_temp.json
✅ 已完成：聯發科台灣擴產
✅ 已完成：聯發科突破季線


In [ ]:
import openai
import json
import time
import os

input_file = '訓練.json'
output_file = '訓練_updated.json'
temp_file = '訓練_temp.json'

# 讀取現有資料
if os.path.exists(temp_file):
    print("🔄 偵測到未完成的 temp 檔案，繼續補完...")
    with open(temp_file, 'r', encoding='utf-8') as file:
        training_data = json.load(file)
else:
    with open(input_file, 'r', encoding='utf-8') as file:
        training_data = json.load(file)

# 擴充主題列表（你可根據需要再加入更多）
topics = [
    "台積電3奈米製程量產", "鴻海電動車進度", "聯發科新一代晶片", "緯創印度廠擴產", "群創面板報價走勢",
    "南亞科記憶體出貨", "華碩AI PC布局", "廣達伺服器訂單", "泰鼎PCB產能", "台達電充電樁業務",
    "技嘉高階主機板出貨", "世界先進產能利用率", "宏碁電競品牌銷售狀況", "南電ABF載板毛利率變化",
    "友達面板稼動率", "欣興PCB封裝新產線", "力積電資本支出", "中美晶太陽能業務擴展", "合晶矽晶圓價格",
    "智邦交換器市佔率", "祥碩高速傳輸晶片營收", "穩懋砷化鎵訂單", "元太電子紙需求", "凌陽電IC設計出貨",
    "技術面：KD黃金交叉搭配爆量紅K", "MACD翻紅並突破0軸", "外資連十日買超超過萬張"
]

# Function schema
function_def = {
    "name": "generate_stock_advice",
    "description": "根據主題生成符合投資建議格式的JSON資料",
    "parameters": {
        "type": "object",
        "properties": {
            "instruct": {
                "type": "string",
                "description": "投資建議的問題與參考資料"
            },
            "output": {
                "type": "string",
                "description": "根據參考資料與問題產生的建議回答"
            }
        },
        "required": ["instruct", "output"]
    }
}

client = openai.OpenAI()
new_entries = []

# 從尚未產生的主題開始
existing_topics = {entry["instruct"] for entry in training_data if "instruct" in entry}
remaining_topics = [t for t in topics if all(t not in et for et in existing_topics)]

for idx, topic in enumerate(remaining_topics):
    try:
        user_prompt = f"""根據主題「{topic}」，請產生一則股市投資建議，包含以下結構：
- instruct 欄位：請列出與主題相關的 4~6 點參考資料，這些資料可以包含基本面資訊（如財報、營收、產能、產業趨勢）與技術分析資訊（如均線、K線、KD、RSI、成交量、壓力支撐、籌碼等），其中至少包含一點技術分析。最後請補上一句投資決策問題。
- output 欄位：請根據上述資料，提供對該股票的投資建議，包含預測與操作策略，建議長度 80~100 字。

請用 function calling 格式回傳。"""

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是資深股市分析師，請精準地根據資料生成投資建議。"},
                {"role": "user", "content": user_prompt}
            ],
            functions=[function_def],
            function_call={"name": "generate_stock_advice"},
            temperature=0.7
        )

        # 提取結果
        response_args = response.choices[0].message.function_call.arguments
        result = json.loads(response_args)
        new_entries.append(result)
        training_data.append(result)

        print(f"✅ 已完成：{topic}")

        # 每3筆儲存一次暫存檔
        if len(new_entries) % 3 == 0 or idx == len(remaining_topics) - 1:
            with open(temp_file, 'w', encoding='utf-8') as file:
                json.dump(training_data, file, ensure_ascii=False, indent=2)
            print("💾 已暫存資料至訓練_temp.json")

        time.sleep(1)

    except Exception as e:
        print(f"⚠️ 生成「{topic}」時發生錯誤: {str(e)}")

# 最後儲存
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(training_data, file, ensure_ascii=False, indent=2)

if os.path.exists(temp_file):
    os.remove(temp_file)

# 顯示新資料
print("\n✅ 所有新生成資料：")
for entry in new_entries:
    print(json.dumps(entry, ensure_ascii=False, indent=2))
